<a href="https://www.kaggle.com/code/avtnshm/sirio-libanes-icu-pred?scriptVersionId=158903863" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import warnings
warnings.simplefilter('ignore')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid19/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx


In [2]:
df = pd.read_excel('/kaggle/input/covid19/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx')
df.head()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,...,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
2,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-6,0
3,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,NaN,NaN,NaN,NaN,-1.000000,-1.000000,6-12,0
4,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-0.238095,-0.818182,-0.389967,0.407558,-0.230462,0.096774,-0.242282,-0.814433,ABOVE_12,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1925 entries, 0 to 1924
Columns: 231 entries, PATIENT_VISIT_IDENTIFIER to ICU
dtypes: float64(225), int64(4), object(2)
memory usage: 3.4+ MB


In [4]:
columns_list = df.columns.tolist()
print(columns_list)

['PATIENT_VISIT_IDENTIFIER', 'AGE_ABOVE65', 'AGE_PERCENTIL', 'GENDER', 'DISEASE GROUPING 1', 'DISEASE GROUPING 2', 'DISEASE GROUPING 3', 'DISEASE GROUPING 4', 'DISEASE GROUPING 5', 'DISEASE GROUPING 6', 'HTN', 'IMMUNOCOMPROMISED', 'OTHER', 'ALBUMIN_MEDIAN', 'ALBUMIN_MEAN', 'ALBUMIN_MIN', 'ALBUMIN_MAX', 'ALBUMIN_DIFF', 'BE_ARTERIAL_MEDIAN', 'BE_ARTERIAL_MEAN', 'BE_ARTERIAL_MIN', 'BE_ARTERIAL_MAX', 'BE_ARTERIAL_DIFF', 'BE_VENOUS_MEDIAN', 'BE_VENOUS_MEAN', 'BE_VENOUS_MIN', 'BE_VENOUS_MAX', 'BE_VENOUS_DIFF', 'BIC_ARTERIAL_MEDIAN', 'BIC_ARTERIAL_MEAN', 'BIC_ARTERIAL_MIN', 'BIC_ARTERIAL_MAX', 'BIC_ARTERIAL_DIFF', 'BIC_VENOUS_MEDIAN', 'BIC_VENOUS_MEAN', 'BIC_VENOUS_MIN', 'BIC_VENOUS_MAX', 'BIC_VENOUS_DIFF', 'BILLIRUBIN_MEDIAN', 'BILLIRUBIN_MEAN', 'BILLIRUBIN_MIN', 'BILLIRUBIN_MAX', 'BILLIRUBIN_DIFF', 'BLAST_MEDIAN', 'BLAST_MEAN', 'BLAST_MIN', 'BLAST_MAX', 'BLAST_DIFF', 'CALCIUM_MEDIAN', 'CALCIUM_MEAN', 'CALCIUM_MIN', 'CALCIUM_MAX', 'CALCIUM_DIFF', 'CREATININ_MEDIAN', 'CREATININ_MEAN', 'CREATI

# Data Pre-Processing

In [5]:
categorical_columns = df.select_dtypes(include=['object']).columns
print("Categorical Columns:", categorical_columns)

print("Number of categories in 'AGE_PERCENTIL':", len(df['AGE_PERCENTIL'].unique()))
print("Number of categories in 'WINDOW':", len(df['WINDOW'].unique()))


Categorical Columns: Index(['AGE_PERCENTIL', 'WINDOW'], dtype='object')
Number of categories in 'AGE_PERCENTIL': 10
Number of categories in 'WINDOW': 5


In [6]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['AGE_PERCENTIL'] = label_encoder.fit_transform(df['AGE_PERCENTIL'])
# using the given encoding in data-card
window_mapping = {'0-2': 0, '2-4': 1, '4-6': 2, '6-12': 3, 'Above-12': 4}
df['WINDOW'] = df['WINDOW'].map(window_mapping)

categorical_cols = df.select_dtypes(include=['object']).columns
print("Remaining Categorical Columns:", categorical_cols)


Remaining Categorical Columns: Index([], dtype='object')


In [7]:
df.fillna(method='ffill', inplace=True)   #filling the missing values using the next or previous entry, as suggested in datacard

In [8]:
df.fillna(method='bfill', inplace=True)


In [9]:
missing_cols_after_fill = df.columns[df.isnull().any()]

if len(missing_cols_after_fill) == 0:
    print("No more missing values in the dataset after filling.")
else:
    for col in missing_cols_after_fill:
        print(f"Column '{col}' still has missing values.")


No more missing values in the dataset after filling.


# RFE for feature Selection

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import pandas as pd

target = 'ICU'
X = df.drop(target, axis=1)
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

feature_importances = pd.Series(rf_classifier.feature_importances_, index=X.columns)

top_features = feature_importances.nlargest(20).index

print("Top 20 Features:")
print(top_features)

rfe_selector = RFE(rf_classifier, n_features_to_select=20)
rfe_selector = rfe_selector.fit(X_train, y_train)

selected_features_rfe = X.columns[rfe_selector.support_]

print("\nTop 20 Features (RFE):")
print(selected_features_rfe)


Top 20 Features:
Index(['WINDOW', 'RESPIRATORY_RATE_MEAN', 'RESPIRATORY_RATE_MEDIAN',
       'RESPIRATORY_RATE_MAX', 'RESPIRATORY_RATE_MIN', 'LACTATE_MAX',
       'LACTATE_MEDIAN', 'BLOODPRESSURE_DIASTOLIC_MIN',
       'BLOODPRESSURE_DIASTOLIC_MEDIAN', 'BLOODPRESSURE_DIASTOLIC_MEAN',
       'LACTATE_MEAN', 'LACTATE_MIN', 'RESPIRATORY_RATE_DIFF', 'AGE_PERCENTIL',
       'RESPIRATORY_RATE_DIFF_REL', 'BLOODPRESSURE_SISTOLIC_MEAN',
       'BLOODPRESSURE_DIASTOLIC_MAX', 'BLOODPRESSURE_SISTOLIC_MEDIAN',
       'BLOODPRESSURE_SISTOLIC_MIN', 'PATIENT_VISIT_IDENTIFIER'],
      dtype='object')

Top 20 Features (RFE):
Index(['AGE_PERCENTIL', 'CALCIUM_MIN', 'CREATININ_MEAN', 'LACTATE_MEDIAN',
       'LACTATE_MIN', 'LEUKOCYTES_MEDIAN', 'LEUKOCYTES_MAX',
       'BLOODPRESSURE_DIASTOLIC_MEAN', 'BLOODPRESSURE_SISTOLIC_MEAN',
       'RESPIRATORY_RATE_MEAN', 'TEMPERATURE_MEAN',
       'BLOODPRESSURE_DIASTOLIC_MEDIAN', 'BLOODPRESSURE_SISTOLIC_MEDIAN',
       'RESPIRATORY_RATE_MEDIAN', 'BLOODPRESSURE_DIAS

# Logistic regression classifier

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
logreg_model = LogisticRegression(random_state=42)
logreg_model.fit(X_train, y_train)
y_pred = logreg_model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2%}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 85.71%

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.95      0.91       278
           1       0.83      0.61      0.70       107

    accuracy                           0.86       385
   macro avg       0.85      0.78      0.80       385
weighted avg       0.85      0.86      0.85       385


Confusion Matrix:
[[265  13]
 [ 42  65]]


# XgBoost

In [12]:
from xgboost import XGBClassifier


X = df[['AGE_PERCENTIL', 'CALCIUM_MIN', 'CREATININ_MEAN', 'LACTATE_MEDIAN',
       'LACTATE_MIN', 'LEUKOCYTES_MEDIAN', 'LEUKOCYTES_MAX',
       'BLOODPRESSURE_DIASTOLIC_MEAN', 'BLOODPRESSURE_SISTOLIC_MEAN',
       'RESPIRATORY_RATE_MEAN', 'TEMPERATURE_MEAN',
       'BLOODPRESSURE_DIASTOLIC_MEDIAN', 'BLOODPRESSURE_SISTOLIC_MEDIAN',
       'RESPIRATORY_RATE_MEDIAN', 'BLOODPRESSURE_DIASTOLIC_MIN',
       'RESPIRATORY_RATE_MIN', 'BLOODPRESSURE_DIASTOLIC_MAX',
       'BLOODPRESSURE_SISTOLIC_MAX', 'RESPIRATORY_RATE_MAX', 'WINDOW']]

y = df['ICU']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_classifier = XGBClassifier()
xgb_classifier.fit(X_train, y_train)

y_pred_xgb = xgb_classifier.predict(X_test)

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
classification_report_xgb = classification_report(y_test, y_pred_xgb)
confusion_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)

print("XGBoost Classifier:")
print(f"Accuracy: {accuracy_xgb * 100:.2f}%\n")
print("Classification Report:\n", classification_report_xgb)
print("Confusion Matrix:\n", confusion_matrix_xgb)


XGBoost Classifier:
Accuracy: 88.57%

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.94      0.92       278
           1       0.83      0.74      0.78       107

    accuracy                           0.89       385
   macro avg       0.87      0.84      0.85       385
weighted avg       0.88      0.89      0.88       385

Confusion Matrix:
 [[262  16]
 [ 28  79]]


# CatBoost

In [13]:
from catboost import CatBoostClassifier


X = df[['AGE_PERCENTIL', 'CALCIUM_MIN', 'CREATININ_MEAN', 'LACTATE_MEDIAN',
       'LACTATE_MIN', 'LEUKOCYTES_MEDIAN', 'LEUKOCYTES_MAX',
       'BLOODPRESSURE_DIASTOLIC_MEAN', 'BLOODPRESSURE_SISTOLIC_MEAN',
       'RESPIRATORY_RATE_MEAN', 'TEMPERATURE_MEAN',
       'BLOODPRESSURE_DIASTOLIC_MEDIAN', 'BLOODPRESSURE_SISTOLIC_MEDIAN',
       'RESPIRATORY_RATE_MEDIAN', 'BLOODPRESSURE_DIASTOLIC_MIN',
       'RESPIRATORY_RATE_MIN', 'BLOODPRESSURE_DIASTOLIC_MAX',
       'BLOODPRESSURE_SISTOLIC_MAX', 'RESPIRATORY_RATE_MAX', 'WINDOW']]

y = df['ICU']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

catboost_classifier = CatBoostClassifier()
catboost_classifier.fit(X_train, y_train)

y_pred_catboost = catboost_classifier.predict(X_test)

accuracy_catboost = accuracy_score(y_test, y_pred_catboost)
classification_report_catboost = classification_report(y_test, y_pred_catboost)
confusion_matrix_catboost = confusion_matrix(y_test, y_pred_catboost)

print("CatBoost Classifier:")
print(f"Accuracy: {accuracy_catboost * 100:.2f}%\n")
print("Classification Report:\n", classification_report_catboost)
print("Confusion Matrix:\n", confusion_matrix_catboost)


Learning rate set to 0.012388
0:	learn: 0.6827027	total: 58.7ms	remaining: 58.6s
1:	learn: 0.6715970	total: 61.8ms	remaining: 30.8s
2:	learn: 0.6616081	total: 64.5ms	remaining: 21.5s
3:	learn: 0.6516550	total: 67.2ms	remaining: 16.7s
4:	learn: 0.6405174	total: 69.7ms	remaining: 13.9s
5:	learn: 0.6324625	total: 72.3ms	remaining: 12s
6:	learn: 0.6230025	total: 74.8ms	remaining: 10.6s
7:	learn: 0.6150998	total: 77.4ms	remaining: 9.6s
8:	learn: 0.6060666	total: 79.9ms	remaining: 8.8s
9:	learn: 0.5984107	total: 82.5ms	remaining: 8.16s
10:	learn: 0.5900217	total: 85.1ms	remaining: 7.65s
11:	learn: 0.5816240	total: 87.9ms	remaining: 7.24s
12:	learn: 0.5734692	total: 90.7ms	remaining: 6.89s
13:	learn: 0.5665849	total: 93.5ms	remaining: 6.58s
14:	learn: 0.5600951	total: 96ms	remaining: 6.3s
15:	learn: 0.5527393	total: 98.7ms	remaining: 6.07s
16:	learn: 0.5462162	total: 102ms	remaining: 5.89s
17:	learn: 0.5397957	total: 104ms	remaining: 5.7s
18:	learn: 0.5337667	total: 107ms	remaining: 5.54s
19:

# Performance of the Models

**Logistic Regression Classifier**:

* Accuracy: 85.71%
* Precision: 86% for class 0, 83% for class 1
* Recall: 95% for class 0, 61% for class 1
* F1-Score: 91% for class 0, 70% for class 1


**XGBoost Classifier**:

* Accuracy: 88.57%
* Precision: 90% for class 0, 83% for class 1
* Recall: 94% for class 0, 74% for class 1
* F1-Score: 92% for class 0, 78% for class 1

 
**CatBoost Classifier**:

* Accuracy: 89.09%
* Precision: 89% for class 0, 88% for class 1
* Recall: 96% for class 0, 70% for class 1
* F1-Score: 93% for class 0, 78% for class 1

 
*CatBoost showed the highest accuracy and balanced performance across precision, recall, and F1-score.*
